# AutoSFX

In [1]:
import importlib
import classify
import scene_understanding
import sync
from PIL import Image
import openai
import os
from openai import OpenAI
import cv2

## Classify

In [7]:
importlib.reload(classify)

<module 'classify' from '/home/s5614279/Master Project/AutoSFX/classify.py'>

In [5]:
running_video = "/home/s5614279/Master Project/AutoSFX/data/video/running.mp4"  # your video path
firework_video = "/home/s5614279/Master Project/AutoSFX/data/video/firework.mp4"
train_video = "/home/s5614279/Master Project/AutoSFX/data/video/train.mp4"
glass_breaking_video = "/home/s5614279/Master Project/AutoSFX/data/video/glass_breaking.mp4"
multivideo = "/home/s5614279/Master Project/AutoSFX/data/video/multivideo2.mp4"
cat = "/home/s5614279/Master Project/AutoSFX/data/video/cat2.mp4"
dog = "/home/s5614279/Master Project/AutoSFX/data/video/dog.mp4"

### Brainstorm with ChatGPT

In [ ]:
importlib.reload(scene_understanding)
# create SceneUnderstanding object
scene_understanding = scene_understanding.SceneUnderstanding()


resized_frame = scene_understanding.process_video(running_video)
#frame_tags = []
#frame_context = []

frame_values = []
frame_objects = []

#output_dir = 'output_frames'
#os.makedirs(output_dir, exist_ok=True)

#for i, frame in enumerate(resized_frame[::15]):
for i, frame in enumerate(resized_frame[::5]):
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    #tags, context = scene_understanding.analyze_image(pil_image)
    #frame_tags.append(tags)
    #frame_context.append(context)

    values, objects = scene_understanding.analyze_image(pil_image)
    frame_score = {

    }
    frame_values.append(values)
    frame_objects.append(objects)

    #image_save_path = os.path.join(output_dir, f'frame_{i:04d}.jpg')
    #pil_image.save(image_save_path)


client = OpenAI()

api_key = os.getenv('OPENAI_API_KEY')

openai.api_key = api_key

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You will be provided with a context describing a scene, and your task is to give the sound suggestions that collectively form the audio landscape of the described scene"},
    {"role": "user", "content": f"What do I hear in the following video description, give me a list: {frame_context[0]}"}
  ]
)

print(completion.choices[0].message)


### Auto Classify

In [7]:
classify = classify.Classify()

resized_frame = classify.process_video(cat)

frame_values = []
frame_objects = []
frame_scores = []

for i, frame in enumerate(resized_frame[::2]):
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    values, objects = classify.recognize_objects(pil_image)
    frame_score = {
        'object': objects, #top 5 
        'score': values
    }
    frame_values.append(values)
    frame_objects.append(objects)
    frame_scores.append(frame_score)


Now you can see the interfer result of every frame.

## Sync

In [3]:
importlib.reload(sync)

<module 'sync' from '/home/s5614279/Master Project/AutoSFX/sync.py'>

In [6]:
syncer = sync.ObjectIntervalSync(train_video)
syncer.analyze_frames()
syncer.calculate_intervals()
intervals = syncer.get_intervals()
object_infos = syncer.get_object_infos()
ambience = syncer.get_ambience()

print(intervals)
print(f"Selected ambience: {ambience}")


{'train': [(1, 74, 2.433333333333333, False)]}
Selected ambience: rainy


We have the intervals, and then we can retrieve the sound effects audio based on the intervals.

## Retrieve

In [9]:
import retrieve_simplify
import IPython.display as ipd
import merge
import importlib

importlib.reload(retrieve_simplify)
importlib.reload(merge)

video_path = "/home/s5614279/Master Project/AutoSFX/data/video/running.mp4"
csv_path = "/home/s5614279/Master Project/AutoSFX/esc50.csv"
audio_folder = "/home/s5614279/Master Project/AutoSFX/data/audio/"
ambience_folder = "/home/s5614279/Master Project/AutoSFX/data/ambience/"
output_folder = "/home/s5614279/Master Project/AutoSFX/output_video/"


matched_effect_audios, ambience_audio, ambience_type = retrieve_simplify.retrieve_audio(video_path, csv_path, audio_folder, ambience_folder)

print(f"Selected ambience: {ambience_type}")
if ambience_audio:
    print(f"Ambience audio file: {ambience_audio['ambience_file']}")
    display(ipd.Audio(ambience_audio['ambience_file']))


if not os.path.exists(output_folder):
    os.makedirs(output_folder)


for obj, audio_lists in matched_effect_audios.items():
    for i, audio_options in enumerate(audio_lists):
        print(f"\nObject: {obj}, Interval {i+1}")
        for j, audio in enumerate(audio_options, 1):
            print(f"{j}. {audio['effect_file']} (duration: {audio['effect_duration']:.2f}s)")
            if audio['needs_fine_sync']:
                for k, start in enumerate(audio['fine_sync_starts']):
                    print(f"   Fine sync start {k+1}: {start:.3f}s")
            display(ipd.Audio(audio['effect_file']))
        
        choice = int(input(f"Choose the best audio for {obj}, Interval {i+1} (1-{len(audio_options)}): ")) - 1
        chosen_audio = audio_options[choice]
        
        output_path = os.path.join(output_folder, f"{obj}_interval_{i+1}_merged.mp4")
        print(f"Merging audio for object: {obj}, Interval {i+1}")
        success = merge.merge_audio_video(video_path, [chosen_audio], ambience_audio, output_path)
        
        if success:
            print(f"Playing merged video: {output_path}")
            display(ipd.Video(output_path))
        else:
            print(f"Failed to merge audio and video for object: {obj}, Interval {i+1}")

Object: walking high_heel, Interval duration: 0.20 seconds
No suitable effect found for object: walking high_heel with duration 0.20
Object: walking high_heel, Interval duration: 2.03 seconds
No suitable effect found for object: walking high_heel with duration 2.03
Selected ambience: windy
Ambience audio file: /home/s5614279/Master Project/AutoSFX/data/ambience/windy.mp3



Object: walking high_heel, Interval 1


ValueError: invalid literal for int() with base 10: ''